#### 排序不等式
设有两数列$a_1, a_2, ..., a_n$和$b_1, b_2, ..., b_n$，满足$a_1 \leq a_2 \leq ... \leq a_n$, $b_1 \leq b_2 \leq ... \leq b_n$，$c_1, c_2, ..., c_n$是$b_1, b_2, ..., b_n$的乱序排列，
则有$\sum\limits_{i=1}^{n}a_ib_i  \geq \sum\limits_{i=1}^{n}a_ic_i \geq \sum\limits_{i=1}^{n}a_ib_{n+1-i}$
例：
若$a_1 \leq a_2$、$b_1 \leq b_2$，
则$a_1b_1 \leq a_2b_2$

**例 道路的最大重要性 T2285**

In [2]:
def maximumImportance(n: int, roads: list[list[int]]) -> int:
    degree = [0 for _ in range(n)]
    for i, j in roads:
        degree[i] += 1
        degree[j] += 1
    res = 0
    for i, j in enumerate(sorted(degree), 1):
        res += i * j
    return res

print(maximumImportance(5, [[0,1],[1,2],[2,3],[0,2],[1,3],[2,4]]))

43


#### 线段树

In [ ]:
# 基本形态
class Tree:
    def __init__(self, n, m):
        self.sum = [0] * (4 * n)

    # 给 idx 下标增加 val
    # self.add(1, 1, n, idx, val)
    def add(self, o: int, l: int, r: int, idx: int, val: int):
        if l == r:
            self.sum[l] += val
            return
        mid = (l + r) // 2
        if idx <= mid:
            self.add(o * 2, l, mid, idx, val)
        else:
            self.add(o * 2 + 1, mid + 1, r, idx, val)
        self.sum[o] = self.sum[o * 2] + self.sum[o * 2 + 1]

    # 返回 [L, R] 范围内的元素和
    # self.query_sum(1, 1, n, L, R)
    def query_sum(self, o: int, l: int, r: int, L: int, R: int):
        if L <= l and r <= R:
            return self.sum[o]
        sum = 0
        mid = (l + r) // 2
        if L <= mid:
            sum += self.query_sum(o * 2, l, mid, L, R)
        if R > mid:
            sum += self.query_sum(o * 2 + 1, mid + 1, r, L, R)
        return sum


### Lazy线段树

In [4]:
class TreeNode:
    """
    lazy: 维护每个区间需要更新的值
          如果这个值 = 0，表示不需要更新
          如果这个值！=0，
            表示需要更新，
            表示更新停在这里，不再递归
            如果又来了一个更新，破坏了lazy，则继续更新
    """
    def __init__(self):
        self.left = -1
        self.right = -1
        self.lazy = 0
        self.sum_num = 0

    def __str__(self):
        return '[%s,%s,%s,%s]' % (self.left, self.right, self.sum_num, self.lazy)
    
    def __repr__(self) -> str:
        return '[%s,%s,%s,%s]' % (self.left, self.right, self.sum_num, self.lazy)

class SegmentTree:
    def __init__(self, n: int):
        self.n = n
        self.max_size = 2 << n.bit_length()  # 空间优化
        self.tree = [TreeNode() for _ in range(self.max_size)]

        self._build(1, 1, n)

    # 向上维护数组：在递归结束向上返回时，将左右孩子的相加到父节点
    def _maintain(self, index: int) -> None:
        self.tree[index].sum_num = self.tree[index*2].sum_num + self.tree[index*2+1].sum_num

    # 把lazy的内容向下传递一层        
    def _push_lazy(self, index: int):  
        if self.tree[index].lazy != 0:
            # 先将其左右孩子的lazy加上自己的lazy，表示lazy的传递
            self.tree[index * 2].lazy += self.tree[index].lazy
            self.tree[index * 2 + 1].lazy += self.tree[index].lazy
            # 更新子树的值
            self.tree[index*2].sum_num = (self.tree[index*2].right - self.tree[index*2].left + 1) * self.tree[index].lazy
            self.tree[index*2+1].sum_num = (self.tree[index*2+1].right - self.tree[index*2+1].left + 1) * self.tree[index].lazy
            # 再清除自己的lazy
            self.tree[index].lazy = 0

    def _build(self, index: int, l: int, r: int) -> None:
        """
        :param index: 节点编号
        :param l: 左端点的位置
        :param r: 右端点的位置
        l 与 r 为闭区间
        """
        # TODO: 维护内容
        self.tree[index].left = l
        self.tree[index].right = r

        if l == r: # 到达底部
            # TODO: 底部操作
            return
        # 递归
        mid = (l + r) // 2
        self._build(index * 2, l, mid)
        self._build(index * 2 + 1, mid + 1, r)
        self._maintain(index)

    def _update(self, index: int, l: int, r: int, ql: int, qr: int, val: int) -> None:
        """
        更新 [ql, qr] 内容
        :param index:
        :param l:
        :param r:
        :param ql: 目标区间左端点
        :param qr: 目标区间右端点
        :param val: 需要加上的数
        """
        # 当前区间已经被目标区间包含 不用递归
        if ql <= l and r <= qr:
            # 不再继续递归
            # 更新和（长度 * 增加的值）
            self.tree[index].sum_num += (r - l + 1) * val
            # 更新lazy标记 直接增加val
            self.tree[index].lazy += val
            return

        # 需要继续递归，则把lazy的内容传下去
        self._push_lazy(index)

        # 递归操作
        mid = (l + r) // 2
        if mid >= ql:  # 目标区间与当前区间的左子树有交集 递归左子树
            self._update(index * 2, l, mid, ql, qr, val)
        if mid + 1 <= qr:
            self._update(index * 2 + 1, mid + 1, r, ql, qr, val)

        # TODO: 维护
        self._maintain(index)

    def update(self, val: int, l: int, r: int = None) -> None:
        self._update(1, 1, self.n, l, l if r is None else r, val)
        
    def _query(self, index: int, l: int, r: int, ql: int, qr: int) -> None:
        # 当前递归的区间已经被包含在目标区间中
        if ql <= l and r <= qr:
            return self.tree[index].sum_num
        
        # 需要继续递归，则把lazy的内容传下去
        self._push_lazy(index)

        mid = (l + r) // 2
        res = 0
        if ql <= mid:  # 目标区间与左子树有交点
            res += self._query(index*2, l, mid, ql, qr)
        if mid+1 <= qr:
            res += self._query(index*2+1, mid+1, r, ql, qr)
        return res

    def query(self, l, r = None):
        return self._query(1, 1, self.n, l, l if r is None else r)



In [5]:
from pprint import pprint as print
t = SegmentTree(4)
t.update(2, 1, 3)
print(t.query(1, 3))

t.update(-1, 3)
print(t.query(3))

6
1


# 例

# [[CRCI2007-2008] PLATFORME 平板](https://www.luogu.com.cn/problem/P2003)

## 题目描述

为了进行一种游戏，现决定搭造一些平板，而各个平板的地址已经选定。基于最普遍的认识，没有任何支持物的平板不可能漂浮在空中。说的更精确些，任意一平板的两端必需有支柱或者它在另一块平板上。

你会得到各个平板在坐标系中的坐标（如左下图）。每一块平板的坐标都是由它的高度（与地板间的垂直距离）和它的水平方位（开始和结束）决定的。每个支柱都距它支撑的平板的边缘半个单位（如图）。

算出支持所有平板的支柱的总长度。

 ![](https://cdn.luogu.com.cn/upload/pic/994.png)

## 输入格式

第一行包括1个整数N，1 ≤ N ≤ 100，即平板的总数。

接下来的N行每行都是一块平板的坐标，是相应的Y，X1和 X2。即高度和水平的边缘坐标。所有的数都是不大于10000的正整数且满足X2 > X1+1（也可这样理解，每一块平板的长度至少为2）。

输入保证任意两块平板间没有重叠部分。

## 输出格式

要撑起所有平板所需的支柱的总长度。

## 样例 #1

### 样例输入 #1

```
3
1 5 10
3 1 5
5 3 7
```

### 样例输出 #1

```
14
```

## 解题思路
可以直接将lazy线段树复制过来
为简化数据结构使用两个数组作为节点
由于柱子偏移了0.5个单位，简单起见直接将$x \times 2$
这样询问时直接询问$x_1 + 1$和$x_2 - 1$即可，同时数组也要开两倍大小

In [3]:
class SegmentTree:
    def __init__(self, n):
        self.n = n
        self.tree = [0] * (2 << n.bit_length())
        self.lazy = [0] * (2 << n.bit_length())

    def update(self, index, ql, qr, val, l, r):
        if ql <= l and r <= qr:
            self.tree[index] = val
            self.lazy[index] = val
            return
        if self.lazy[index] != 0:
            lazy = self.lazy[index]
            self.lazy[index*2] = lazy
            self.lazy[index*2+1] = lazy
            self.tree[index*2] = lazy
            self.tree[index*2+1] = lazy
            self.lazy[index] = 0
        mid = (l + r) // 2
        if ql <= mid:
            self.update(index*2, ql, qr, val, l, mid)
        if mid + 1<= qr:
            self.update(index*2+1, ql, qr, val, mid+1, r)
        self.tree[index] = self.tree[index*2]

    def query(self, index, ql, qr, l, r):
        if ql <= l and r <= qr:
            return self.tree[index]
        if self.lazy[index] != 0:
            lazy = self.lazy[index]
            self.lazy[index*2] = lazy
            self.lazy[index*2+1] = lazy
            self.tree[index*2] = lazy
            self.tree[index*2+1] = lazy
            self.lazy[index] = 0
        mid = (l + r) // 2
        res = 0
        if ql <= mid:
            res = self.query(index*2, ql, qr, l, mid)
        if mid + 1<= qr:
            res = self.query(index*2+1, ql, qr, mid+1, r)
        return res


n = int(input())
board = []
for _ in range(n):
    board.append(tuple(map(int, input().split())))
board.sort(key=lambda x: x[0])

tree = SegmentTree(20000)

res = 0
for y, x1, x2 in board:
    res += y - tree.query(1, x1*2+1, x1*2+1, 1, tree.n)
    res += y - tree.query(1, x2*2-1, x2*2-1, 1, tree.n)
    tree.update(1, x1*2, x2*2, y, 1, tree.n)

print(res)


ValueError: invalid literal for int() with base 10: ''